**SCRAPPING**

Dans cette première étape, nous allons effectuer du scrapping sur le site hellowork

Celui-ci référence les emplois, le salaires, les zones sur une seule et même page.

Il n'est pas nécéssaire de changer de page pour afficher plus d'offres, les emplois s'actualisent automatiquement en scrollant en bas de page.

**Ce code a donc pour but de :**
- Se rendre sur le site hellowork.com, section Data scientist en France
- Collecter les données et les ranger dans un tableau via pandas (Nom des posts, nom de l'entreprise, type de post, zones, salaires, mois en cours, années en cours)
- Passer à la page suivante dès que le scrapping est fait
- Créer un fichier Excel avec les données collectées

Vous pouvez run all directement

In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
from datetime import datetime

def extract_salary(salary_str):
    numbers = re.findall(r'\d+', salary_str)
    return ''.join(numbers) if numbers else None

def process_salary(salary_str):
    if salary_str and '-' in salary_str:
        salary_min, salary_max = salary_str.split('-')
        return int(extract_salary(salary_min)), int(extract_salary(salary_max))
    else:
        return None, None

# Initialisation du pilote Selenium
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

current_page = 1
maximum_pages = 6
data = []

# Collecte des données
# ... [Le reste du code est inchangé] ...

# Collecte des données
while current_page <= maximum_pages:
    url = f'https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data+scientist&ray=all&d=all&msa=21000&p={current_page}'
    driver.get(url)
    time.sleep(3)

    job_listings = driver.find_elements(By.CSS_SELECTOR, 'section[data-controller="teleport"]')

    for job in job_listings:
        # Sélecteurs CSS pour chaque élément
        job_title_elements = job.find_elements(By.CSS_SELECTOR, 'a[class="md:tw-text-xlOld tw-text-2xlOld tw-leading-[1.625rem]"]')
        contract_elements = job.find_elements(By.CSS_SELECTOR, 'span[data-cy="contract"]')
        company_elements = job.find_elements(By.CSS_SELECTOR, 'span.tw-mr-2')
        location_elements = job.find_elements(By.CSS_SELECTOR, 'span.tw-text-ellipsis')
        salary_elements = job.find_elements(By.CSS_SELECTOR, 'span.tw-text-jobsCandidacy.tw-typo-s')

        # Extraction des données
        job_title = job_title_elements[0].text if job_title_elements else 'N/A'
        contract = contract_elements[0].text if contract_elements else 'N/A'
        company = company_elements[0].text if company_elements else 'N/A'
        location = location_elements[0].text if location_elements else 'N/A'
        salary = salary_elements[0].text if salary_elements else None
        salary_min, salary_max = process_salary(salary)
        data.append([job_title, contract, company, location, salary_min, salary_max])
        print(f"Titre: {job_title}, Entreprise: {company}, Lieu: {location}, Salairemax: {salary_max}, salairemin : {salary_min}")  

    current_page += 1

driver.quit()

# Création du DataFrame avec pandas
df = pd.DataFrame(data, columns=["Job Title", "Contrat", "Company", "Location", "Salary min", "Salary max"])

# Nettoyage et conversion de 'Salary min' et 'Salary max' en valeurs numériques
df['Salary min'] = df['Salary min'].replace(r'[^\d.]+', '', regex=True)
df['Salary max'] = df['Salary max'].replace(r'[^\d.]+', '', regex=True)
df['Salary min'] = pd.to_numeric(df['Salary min'], errors='coerce')
df['Salary max'] = pd.to_numeric(df['Salary max'], errors='coerce')

# Calculez la moyenne des salaires min et max
df['Salary'] = df[['Salary min', 'Salary max']].mean(axis=1, skipna=True)

# Suppression des lignes où "Salary" est vide (NaN)
df = df.dropna(subset=['Salary'])

# Enregistrement dans un fichier Excel
df.to_excel('JobsList.xlsx', index=False)




Titre: Data Scientist H/F, Entreprise: Bouygues Telecom, Lieu: Meudon - 92, Salairemax: 55000, salairemin : 45000
Titre: Data Enginer H/F, Entreprise: HR Team, Lieu: Nice - 06, Salairemax: 48000, salairemin : 37000
Titre: Business Developer - Full Remote - DevOps Infra et Cybersécurité H/F, Entreprise: UPTOO, Lieu: Marseille - 13, Salairemax: 70000, salairemin : 45000
Titre: Développeur React.Js Confirmé Startup IA H/F, Entreprise: Seyos, Lieu: Nantes - 44, Salairemax: 55000, salairemin : 45000
Titre: Data Engineer H/F, Entreprise: Crédit Agricole Champagne Bourgogne, Lieu: Dijon - 21, Salairemax: 55000, salairemin : 45000
